In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import get_blas_funcs
from scipy.special import expit
import time
import csv
import threadpoolctl
from threadpoolctl import threadpool_limits

def simplify_col(table, column):
    vals = table[column].unique()
    val_to_id = dict()
    for i, val in enumerate(vals):
        val_to_id[val] = i
    table[column] = [val_to_id[v] for v in table[column]]

def one_hot_encode_col(table, column):
    # Get one hot encoding of columns B
    one_hot = pd.get_dummies(table[column], dtype=float)
    # Drop column B as it is now encoded
    table = table.drop(column,axis = 1)
    # Join the encoded df
    table = table.join(one_hot)
    return table

def one_hot_encode_cols(table, columns):
    for column in columns:
        table = one_hot_encode_col(table, column)
    return table

def main():
    with threadpool_limits(limits=1, user_api='blas'):
        customer = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/customer.tbl", sep="|", names=["CustomerKey", "Name", "Address", "NationKey", "Phone", "AcctBal", "MktSegment", "Comment", "Col9"])
        lineitem = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/lineitem.tbl", sep="|", names=["OrderKey", "PartKey", "SuppKey", "LineNumber", "Quantity", "ExtendedPrice", "Discount", "Tax", "ReturnFlag", "LineStatus", "ShipDate", "CommitDate", "ReceiptDate", "ShipInstruct", "ShipMode", "Comment", "Col9"])
        lineitem["LineItemKey"] = range(1, len(lineitem)+1) 
        orders = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/orders.tbl", sep="|", names=["OrderKey", "CustomerKey", "OrderStatus", "TotalPrice", "OrderDate", "OrderPriority", "Clerk", "ShipPriority", "Comment", "Extra"])
        partsupp = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/partsupp.tbl", sep="|", names=["PartKey", "SuppKey", "AvailQty", "SupplyCost", "Comment", "Col9"])
        part = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/part.tbl", sep="|", names=["PartKey", "Name", "MFGR", "Brand", "Type", "Size", "Container", "RetailPrice", "Comment", "Col9"])
        supplier = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/supplier.tbl", sep="|", names=["SuppKey", "Name", "Address", "NationKey", "Phone", "AcctBal", "Comment", "Col9"])

        simplify_col(lineitem, "OrderKey")
        simplify_col(lineitem, "PartKey")
        simplify_col(lineitem, "SuppKey")
        simplify_col(orders, "OrderKey")
        simplify_col(orders, "CustomerKey")
        simplify_col(customer, "CustomerKey")
        simplify_col(part, "PartKey")
        simplify_col(supplier, "SuppKey")

        lineitem = lineitem[["LineItemKey", "OrderKey", "PartKey", "SuppKey"]]
        orders = orders[["OrderKey", "CustomerKey", "OrderStatus", "TotalPrice", "OrderPriority", "ShipPriority"]]
        orders = one_hot_encode_cols(orders, ["OrderStatus", "OrderPriority", "ShipPriority"])
        customer = customer[["CustomerKey", "NationKey", "AcctBal", "MktSegment"]]
        customer = one_hot_encode_cols(customer, ["NationKey", "MktSegment"])
        partsupp = partsupp[["PartKey", "SuppKey"]]
        supplier = supplier[["SuppKey", "NationKey", "AcctBal"]]
        supplier = one_hot_encode_cols(supplier, ["NationKey"])
        part = part[["PartKey", "MFGR", "Brand", "Size", "Container", "RetailPrice"]]
        part = one_hot_encode_cols(part, ["MFGR", "Brand", "Container"])
        theta = np.random.rand(144)
        theta_s = pd.Series(theta)
        star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
        theta_s.index = star_join.columns
        self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
        theta_sj = np.random.rand(133)
        theta_sj_s = pd.Series(theta_sj)
        pd_lr_time = 0
        pd_log_time = 0
        pd_cov_time = 0
        pd_lr_time2 = 0
        pd_log_time2 = 0
        pd_cov_time2 = 0
        np_lr_time = 0
        np_log_time = 0
        np_cov_time = 0
        np_lr_time2 = 0
        np_log_time2 = 0
        np_cov_time2 = 0
        blas_lr_time = 0
        blas_log_time = 0
        blas_cov_time = 0
        blas_lr_time2 = 0
        blas_log_time2 = 0
        blas_cov_time2 = 0
        n = 3
        for _ in range(1, n + 1):
            pd_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            pd_result = star_join.dot(theta_s)
            pd_end = time.time()
            pd_lr_time = pd_lr_time + pd_end - pd_start
            print(pd_end-pd_start)

            pd_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            pd_result = expit(star_join.dot(theta_s))
            pd_end = time.time()
            pd_log_time = pd_log_time + pd_end-pd_start
            print(pd_end-pd_start)

            pd_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            pd_result = star_join.cov()
            pd_end = time.time()
            pd_cov_time = pd_cov_time + pd_end-pd_start
            print(pd_end-pd_start)

            pd_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            pd_result = self_join.dot(theta_sj_s)
            pd_end = time.time()
            pd_lr_time2 = pd_lr_time + pd_end - pd_start
            print(pd_end-pd_start)

            pd_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            pd_result = expit(self_join.dot(theta_sj_s))
            pd_end = time.time()
            pd_log_time2 = pd_log_time + pd_end-pd_start
            print(pd_end-pd_start)

            pd_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            pd_result = self_join.cov()
            pd_end = time.time()
            pd_cov_time2 = pd_cov_time + pd_end-pd_start
            print(pd_end-pd_start)

            np_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            np_result = np.matmul(X, theta)
            np_end = time.time()
            np_lr_time = np_lr_time + np_end-np_start
            print(np_end-np_start)

            np_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            np_result = expit(np.matmul(X, theta))
            np_end = time.time()
            np_log_time = np_log_time + np_end-np_start
            print(np_end-np_start)

            np_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            np_result = np.matmul(X.T, X)
            np_end = time.time()
            np_cov_time = np_cov_time + np_end-np_start
            print(np_end-np_start)

            np_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            np_result = np.matmul(X, theta_sj)
            np_end = time.time()
            np_lr_time2 = np_lr_time + np_end-np_start
            print(np_end-np_start)

            np_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            np_result = expit(np.matmul(X, theta_sj))
            np_end = time.time()
            np_log_time2 = np_log_time + np_end-np_start
            print(np_end-np_start)

            np_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            np_result = np.matmul(X.T, X)
            np_end = time.time()
            np_cov_time2 = np_cov_time + np_end-np_start
            print(np_end-np_start)

            gemv = get_blas_funcs("gemv", [X, theta])
            blas_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            blas_result = gemv(1, X, theta)
            blas_end = time.time()
            blas_lr_time = blas_lr_time + blas_end - blas_start
            print(blas_end-blas_start)

            gemv = get_blas_funcs("gemv", [X, theta])
            blas_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            blas_result = gemv(1, X, theta)
            blas_end = time.time()
            blas_log_time = blas_log_time + blas_end - blas_start
            print(blas_end-blas_start)

            gemm = get_blas_funcs("gemm", [X.T, X])
            blas_start = time.time()
            star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
            X = star_join.to_numpy()
            blas_result = gemm(1, X.T, X)
            blas_end = time.time()
            blas_cov_time = blas_cov_time + blas_end - blas_start
            print(blas_end-blas_start)

            gemv = get_blas_funcs("gemv", [X, theta_sj])
            blas_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            blas_result = gemv(1, X, theta_sj)
            blas_end = time.time()
            blas_lr_time = blas_lr_time + blas_end - blas_start
            print(blas_end-blas_start)

            gemv = get_blas_funcs("gemv", [X, theta_sj])
            blas_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            blas_result = gemv(1, X, theta_sj)
            blas_end = time.time()
            blas_log_time = blas_log_time + blas_end - blas_start
            print(blas_end-blas_start)

            gemm = get_blas_funcs("gemm", [X.T, X])
            blas_start = time.time()
            self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))
            X = self_join.to_numpy()
            blas_result = gemm(1, X.T, X)
            blas_end = time.time()
            blas_cov_time = blas_cov_time + blas_end - blas_start
            print(blas_end-blas_start)


        pd_lr_time = pd_lr_time / n
        pd_log_time = pd_log_time / n
        pd_cov_time = pd_cov_time / n
        pd_lr_time2 = pd_lr_time2 / n
        pd_log_time2 = pd_log_time2 / n
        pd_cov_time2 = pd_cov_time2 / n
        np_lr_time = np_lr_time / n
        np_log_time = np_log_time / n
        np_cov_time = np_cov_time / n
        np_lr_time2 = np_lr_time2 / n
        np_log_time2 = np_log_time2 / n
        np_cov_time2 = np_cov_time2 / n
        blas_lr_time = blas_lr_time / n
        blas_log_time = blas_log_time / n
        blas_cov_time = blas_cov_time / n
        blas_lr_time2 = blas_lr_time2 / n
        blas_log_time2 = blas_log_time2 / n
        blas_cov_time2 = blas_cov_time2 / n
        data = [["Algorithm", "Method", "ExecuteTime", "OptTime"],
                ["Pandas", "Linear Regression (SQ)", pd_lr_time, 0],
                ["Pandas", "Logistic Regression (SQ)", pd_log_time, 0],
                ["Pandas", "Covariance (SQ)", pd_cov_time, 0],
                ["Numpy", "Linear Regression (SQ)", np_lr_time, 0],
                ["Numpy", "Logistic Regression (SQ)", np_log_time, 0],
                ["Numpy", "Covariance (SQ)", np_cov_time, 0],
                ["BLAS", "Linear Regression (SQ)", blas_lr_time, 0],
                ["BLAS", "Logistic Regression (SQ)", blas_log_time, 0],
                ["BLAS", "Covariance (SQ)", blas_cov_time, 0]]
        with open('/local1/kdeeds/Galley/Experiments/Results/tpch_inference_python.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(data)
        print("Pandas LR", pd_lr_time)
        print("Pandas Log", pd_log_time)
        print("Pandas Cov", pd_cov_time)
        print("Numpy LR", np_lr_time)
        print("Numpy Log", np_log_time)
        print("Numpy Cov", np_cov_time)
        print("BLAS LR", blas_lr_time)
        print("BLAS Log", blas_log_time)
        print("BLAS Cov", blas_cov_time)

In [4]:
customer = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/customer.tbl", sep="|", names=["CustomerKey", "Name", "Address", "NationKey", "Phone", "AcctBal", "MktSegment", "Comment", "Col9"])
lineitem = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/lineitem.tbl", sep="|", names=["OrderKey", "PartKey", "SuppKey", "LineNumber", "Quantity", "ExtendedPrice", "Discount", "Tax", "ReturnFlag", "LineStatus", "ShipDate", "CommitDate", "ReceiptDate", "ShipInstruct", "ShipMode", "Comment", "Col9"])
lineitem["LineItemKey"] = range(1, len(lineitem)+1) 
orders = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/orders.tbl", sep="|", names=["OrderKey", "CustomerKey", "OrderStatus", "TotalPrice", "OrderDate", "OrderPriority", "Clerk", "ShipPriority", "Comment", "Extra"])
partsupp = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/partsupp.tbl", sep="|", names=["PartKey", "SuppKey", "AvailQty", "SupplyCost", "Comment", "Col9"])
part = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/part.tbl", sep="|", names=["PartKey", "Name", "MFGR", "Brand", "Type", "Size", "Container", "RetailPrice", "Comment", "Col9"])
supplier = pd.read_csv("/local1/kdeeds/Galley/Experiments/Data/TPCH/supplier.tbl", sep="|", names=["SuppKey", "Name", "Address", "NationKey", "Phone", "AcctBal", "Comment", "Col9"])

simplify_col(lineitem, "OrderKey")
simplify_col(lineitem, "PartKey")
simplify_col(lineitem, "SuppKey")
simplify_col(orders, "OrderKey")
simplify_col(orders, "CustomerKey")
simplify_col(customer, "CustomerKey")
simplify_col(part, "PartKey")
simplify_col(supplier, "SuppKey")

lineitem = lineitem[["LineItemKey", "OrderKey", "PartKey", "SuppKey"]]
orders = orders[["OrderKey", "CustomerKey", "OrderStatus", "TotalPrice", "OrderPriority", "ShipPriority"]]
orders = one_hot_encode_cols(orders, ["OrderStatus", "OrderPriority", "ShipPriority"])
customer = customer[["CustomerKey", "NationKey", "AcctBal", "MktSegment"]]
customer = one_hot_encode_cols(customer, ["NationKey", "MktSegment"])
partsupp = partsupp[["PartKey", "SuppKey"]]
supplier = supplier[["SuppKey", "NationKey", "AcctBal"]]
supplier = one_hot_encode_cols(supplier, ["NationKey"])
part = part[["PartKey", "MFGR", "Brand", "Size", "Container", "RetailPrice"]]
part = one_hot_encode_cols(part, ["MFGR", "Brand", "Container"])
theta = np.random.rand(144)
W1 = np.random.rand(144, 25)
theta_s = pd.Series(theta)
star_join = lineitem.merge(orders, on="OrderKey").merge(customer, on="CustomerKey").merge(supplier, on="SuppKey").merge(part, on="PartKey")
X = star_join.to_numpy()
#theta_s.index = star_join.columns

(600572, 25)


In [8]:
print(np.matmul(X, W1).shape)

(600572, 25)


In [3]:
self_join = lineitem.merge(lineitem, on="PartKey").merge(part, on="PartKey").merge(supplier, left_on="SuppKey_x", right_on="SuppKey", suffixes=("_1", "_2")).merge(supplier, left_on="SuppKey_y", right_on="SuppKey", suffixes=("_3", "__4"))

In [11]:
theta_sj = np.random.rand(133)
X = self_join.to_numpy()
np_result = np.matmul(X.T, X)